In [1]:
import pandas as pd
import re

## SNOMED to phecode

This is necessary because MSDW diagnoses are encoded as SNOMED codes.

In [150]:
p1 = pd.read_csv('./Resources/phecodeX_unrolled_ICD_CM.csv')
p2 = pd.read_csv('./Resources/phecodeX_unrolled_ICD_WHO.csv')
comb = pd.concat([p1,p2])
comb = comb.rename({'ICD':'code'},axis=1)
comb.loc[comb['vocabulary_id'].str.contains('ICD9'), 'icd_type'] = 'ICD9'
comb.loc[comb['vocabulary_id'].str.contains('ICD10'), 'icd_type'] = 'ICD10'
comb['code'] = comb['code'].astype(str)

comb_null = comb.copy()
comb_null['code'] = comb_null['code'].str.replace('.','')

hp = pd.read_pickle('./Resources/hpo_phecodeX.pkl')
hp['code'] = hp['id'].str.split('_').str[1].astype(str)
hp = hp[['phecode','code']]

snomed = pd.read_pickle('./Resources/snomed_20230901.pkl')
snomed = snomed.merge(comb[['phecode','code']], left_on='mapTarget', right_on='code')
snomed = snomed[['referencedComponentId','phecode']].drop_duplicates().set_axis(['concept_code','phecode'],axis=1)

umls = pd.read_pickle('./Resources/umls.pkl')[[0,11,13]]
umls_snomed = umls.loc[umls[11].str.contains('SNOMED')][[0,13]].set_axis(['UMLS','concept_code'],axis=1)
umls_hpo = umls.loc[umls[11] == 'HPO'][[0,13]].set_axis(['UMLS','code'],axis=1)
umls_hpo['code'] = umls_hpo['code'].str.replace('HP:','').astype(str)
umls_hpo = umls_hpo.merge(hp)
umls = umls_hpo.merge(umls_snomed)[['concept_code','phecode']]

sp = pd.concat([snomed,umls]).drop_duplicates()
sp.to_pickle('./Conversion/snomed_phecode.pkl')


In [171]:
ms = pd.read_csv('./Conversion/msdw_unique_snomed.csv')
ms['concept_code'] = ms['concept_code'].astype(str)

sp = pd.read_pickle('./Conversion/snomed_phecode.pkl')
sp['concept_code'] = sp['concept_code'].astype(str)

rd = pd.read_excel('./Resources/combined_phecodex_list.xlsx')
rd = rd.loc[rd['Type'] == 'Rare']

sp = sp.loc[sp['concept_code'].isin(ms['concept_code'])]
sp = sp.loc[sp['phecode'].isin(rd['phecode'])]

p1 = pd.read_csv('./Resources/phecodeX_unrolled_ICD_CM.csv')
p2 = pd.read_csv('./Resources/phecodeX_unrolled_ICD_WHO.csv')
comb = pd.concat([p1,p2])
comb = comb.rename({'ICD':'code'},axis=1)
comb.loc[comb['vocabulary_id'].str.contains('ICD9'), 'icd_type'] = 'ICD9'
comb.loc[comb['vocabulary_id'].str.contains('ICD10'), 'icd_type'] = 'ICD10'
comb['code'] = comb['code'].astype(str)
comb = comb.loc[comb['phecode'].isin(rd['phecode'])]
comb = comb.loc[comb['vocabulary_id'].str.contains('ICD10')]

all_codes = sp['concept_code'].to_list() + comb['code'].to_list()
all_codes = list(set(all_codes))

with open('./Conversion/rare_phecode_snomed_codes.txt', 'w') as f:
    f.write(f"condition_concept_code IN ({','.join(f'\'{code}\'' for code in all_codes)})")


In [ ]:
# Get diagnoses from MSDW (SQL scripts)
# Diagnoses are in two tables (condition_occurrence and past_medical_history_xtn)
# These tables are too large -> after initial query completes, I further filter on condition_concept_code IN ...
# See above: "f.write(f"condition_concept_code IN ({','.join(f'\'{code}\'' for code in all_codes)})")"

SELECT 
    co.person_id, 
    co.condition_concept_code, 
    co.condition_concept_name, 
    co.condition_source_value, 
    co.condition_start_date
FROM 
    cdm_deid.condition_occurrence co

SELECT 
    co.person_id, 
    co.observation_concept_code, 
    co.observation_concept_name, 
    co.observation_source_value, 
    co.observation_date
FROM 
    cdm_deid.past_medical_history_xtn co


In [185]:
co = pd.read_csv('./OMOP/msdw_selected_phecode_co.csv')
sp['concept_code'] = sp['concept_code'].astype(str)
co['condition_concept_code'] = co['condition_concept_code'].astype(str)
co = co.merge(sp, left_on='condition_concept_code', right_on='concept_code', how='left')
co_icd = co.loc[co['phecode'].isna()]
co = co.loc[co['phecode'].notna()]
co_icd = co_icd.merge(comb, left_on='condition_concept_code', right_on='code')
co = pd.concat([co,co_icd])[['person_id','phecode','condition_start_date']].dropna()
co['condition_start_date'] = pd.to_datetime(co['condition_start_date'])
co = co.sort_values('condition_start_date', ascending=True).drop_duplicates(['person_id','phecode'])
co.to_pickle('./OMOP/msdw_selected_phecode_first.pkl')


/tmp/ipykernel_25676/3261347658.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  co = pd.read_csv('./OMOP/msdw_selected_phecode_co.csv')


## Drug to RxNorm

Merging drug labels by name and RxNorm ID

In [ ]:
# Get unique drugs from MSDW (SQL scripts)

SELECT DISTINCT drug_concept_id, drug_concept_code, drug_concept_name, xtn_drug_generic_name, xtn_generic_ingredient_source_concept_id, xtn_generic_ingredient_source_concept_name  
FROM cdm_deid.drug_exposure

SELECT DISTINCT drug_concept_id, drug_concept_code, drug_concept_name, xtn_drug_generic_name, xtn_generic_ingredient_source_concept_id, xtn_generic_ingredient_source_concept_name  
FROM cdm_deid.medication_administration_xtn


In [2]:
a = pd.read_csv('./OMOP/msdw_drug_exposure_unique_drugs.csv')
b = pd.read_csv('./OMOP/msdw_med_admin_unique_drugs.csv')
drugs = pd.concat([a,b]).drop_duplicates().astype(str)

conv = pd.read_csv('./Conversion/omop_rxnorm.csv', sep='\t', usecols=['concept_id','concept_code','vocabulary_id'])
conv = conv.loc[conv['vocabulary_id'] == 'RxNorm'][['concept_id','concept_code']].astype(int)
conv = conv.set_axis(['drug_concept_id','drug_concept_code'],axis=1).astype(str)
conv['rxnorm'] = 1

drugs = drugs.merge(conv, how='left')

#####

rxnorm = drugs.loc[drugs['rxnorm'] == 1]

umls = pd.read_pickle('./Resources/umls.pkl')[[0,11,13]]

umls_rxnorm = umls.loc[umls[11] == 'RXNORM']
umls_rxnorm = umls_rxnorm[[0,13]].set_axis(['UMLS','drug_concept_code'],axis=1)
umls_rxnorm['drug_concept_code'] = umls_rxnorm['drug_concept_code'].astype(int).astype(str)

umls_drugbank = umls.loc[umls[11] == 'DRUGBANK']
umls_drugbank = umls_drugbank[[0,13]].set_axis(['UMLS','drugbank'],axis=1)

rxnorm = rxnorm.merge(umls_rxnorm).merge(umls_drugbank).drop_duplicates()
#rxnorm['drugbank'].drop_duplicates().to_csv('./Conversion/drugbank_ids.csv', index=None, header=None)

#####

dc = pd.read_csv('./Conversion/drugbank_chembl.txt', header=None, sep='\t')
dc = dc.set_axis(['drugbank','chembl'],axis=1)

rxnorm = rxnorm.merge(dc, how='left')
rxnorm = rxnorm[['drug_concept_id','drugbank','chembl']].drop_duplicates().rename({'chembl':'chembl_id'},axis=1)
rxnorm.to_pickle('./Conversion/omop_drugbank_chembl.pkl')


/tmp/ipykernel_981367/720314136.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  conv = pd.read_csv('./Conversion/omop_rxnorm.csv', sep='\t', usecols=['concept_id','concept_code','vocabulary_id'])


In [3]:
dn = pd.read_csv('./Conversion/drugbank_links.csv')[['DrugBank ID','Name']]
cn = pd.read_csv('./Resources/chembl_34_drug_targets.csv')[['chembl_id','pref_name']]
cn2 = pd.read_csv('./Resources/chembl_missing_names.tsv', sep='\t')[['ChEMBL ID','Name']]

a = pd.read_csv('./OMOP/msdw_drug_exposure_unique_drugs.csv')
b = pd.read_csv('./OMOP/msdw_med_admin_unique_drugs.csv')
drugs = pd.concat([a,b]).drop_duplicates().astype(str)

def clean_m_name(m_name):
    modifiers = ['sulfate', 'sodium', 'hydrochloride', 'hydroxide', 'phosphate', 'tablet', 'injection',
                 'acetate', 'citrate', 'bromide', 'iodide', 'carbonate', 'chloride', 
                 'silicate', 'liposome', 'ultramicrosize', 'isopropyl', 'alcohol',
                 'injectable', 'product', 'powder', 'prefilled', 'syringe', 'tosylate',
                 'mesylate', 'mesylates', 'tartrate', 'decanoate', 'lauroxil', 'magnesium', 'calcium',
                 'dipotassium', 'potassium', 'diproprionate', 'palmitate', 'succinate', 'pamoate', 'valerate',
                 'aspartate', 'strontium', 'epolamine', 'polyethylene', 'proprionate',
                 'maleate','besylate','fumarate','dihydrate','malate','monohydrate',
                 'mertansine','acetonide','lactate','benzoate','propionate','purified',
                 'recombinant','synthetic','semisynthetic','preservative','investigational','unclassified']
    return ' '.join(word for word in m_name.split() if not any(modifier in word.lower() for modifier in modifiers))

def clean_m_name_full(m_name):
    modifiers = ['hr', 'ml', 'mcg', 'pegol', 'alfa', 'susp', 'cal', 'oxide', 'oral', 
                 'zinc', 'pork', 'beef', 'glycol', '%', 'nacl', 'usp', 'sod', 'hbr',
                 'liquid','unt','topical','ointment', 'human']
    pattern = r'\b(?:' + '|'.join(modifiers) + r')\b'
    return re.sub(pattern, '', m_name, flags=re.IGNORECASE).strip()

for col in ['drug_concept_name','xtn_drug_generic_name','xtn_generic_ingredient_source_concept_name']:
    drugs[col] = drugs[col].str.lower().str.replace('hcl','hydrochloride')
    drugs[col] = drugs[col].apply(lambda x: x.split(' mg')[0] if ' mg' in x else x)
    drugs[col] = drugs[col].str.replace(r'\d+\.?\d*', '', regex=True)
    drugs[col] = drugs[col].apply(clean_m_name).apply(clean_m_name_full)

drugs.to_pickle('./Conversion/drug_concept_names.pkl')

drugs = drugs.drop(['drug_concept_code'],axis=1)
drugs = drugs.loc[~drugs['xtn_generic_ingredient_source_concept_name'].str.contains('no matching concept|unspecified', case=False)]
drugs = pd.melt(drugs, id_vars=['drug_concept_id','xtn_generic_ingredient_source_concept_id']).rename({'value':'dc_name'},axis=1).dropna()
drugs = drugs.loc[~drugs['dc_name'].str.contains('no matching concept|unspecified', case=False)]

dn = dn.set_axis(['drugbank','dc_name'],axis=1)
cn = cn.set_axis(['chembl_id','dc_name'],axis=1)
cn2 = cn2.set_axis(['chembl_id','dc_name'],axis=1)
dc = pd.concat([dn,cn,cn2])
dc['dc_name'] = dc['dc_name'].astype(str).str.lower().apply(clean_m_name).apply(clean_m_name_full)

drugs['dc_name'] = drugs['dc_name'].str.replace('{ (','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.replace(') (','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.replace('%','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.replace('/ns','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.replace('/pf','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.replace(',','',regex=False)
drugs['dc_name'] = drugs['dc_name'].str.strip()

drugs = drugs.merge(dc)
drugs = drugs.loc[drugs['dc_name'].str.len() > 2]
drugs = drugs.loc[drugs['dc_name'] != 'nan']

id_name = drugs[['xtn_generic_ingredient_source_concept_id','dc_name']].set_axis(['concept_id','name'],axis=1)
id_name.to_pickle('./Conversion/concept_id_names.pkl')

rxnorm = pd.read_pickle('./Conversion/omop_drugbank_chembl.pkl')
temp = drugs.loc[drugs['xtn_generic_ingredient_source_concept_id'] == '0'][['drug_concept_id','drugbank','chembl_id']].drop_duplicates()
temp = temp.loc[temp['drug_concept_id'] != '0']
rxnorm = pd.concat([rxnorm,temp]).drop_duplicates()
rxnorm.to_pickle('./Conversion/omop_drugbank_chembl.pkl')

drugs = drugs[['xtn_generic_ingredient_source_concept_id','dc_name','drugbank','chembl_id']].drop_duplicates()
drugs.to_pickle('./Conversion/name_drugbank_chembl.pkl')

## Repurposed drugs

In [4]:
dr = pd.read_pickle('./Resources/all_predictions_cov_ot_mantis_cc.pkl').sort_values('prediction', ascending=False)

targets = pd.read_pickle('./Resources/targets.pkl')
targets = targets.drop('CID',axis=1).set_axis(['moa','chembl','gene','drugbank'],axis=1).drop_duplicates()

a = pd.read_pickle('./Conversion/omop_drugbank_chembl.pkl')
a1 = a.merge(targets[['moa','gene','chembl']].dropna().drop_duplicates(), how='left')[['drug_concept_id','moa','gene']]
a2 = a.merge(targets[['moa','gene','drugbank']].dropna().drop_duplicates(), how='left')[['drug_concept_id','moa','gene']]
a = pd.concat([a1,a2])
a = a.loc[a['gene'].notna()].drop_duplicates()
a['moa_priority'] = a['moa'].map({'Negative modulator':2,'Positive modulator':3,'Other':1}).fillna(0)
a = a.sort_values('moa_priority',ascending=False).drop_duplicates(['drug_concept_id','gene'])
a = a.sort_values(['drug_concept_id'])

b = pd.read_pickle('./Conversion/name_drugbank_chembl.pkl').rename({'chembl_id':'chembl'},axis=1)
b1 = b.merge(targets[['moa','gene','chembl']].dropna().drop_duplicates(), how='left')[['xtn_generic_ingredient_source_concept_id','moa','gene']]
b2 = b.merge(targets[['moa','gene','drugbank']].dropna().drop_duplicates(), how='left')[['xtn_generic_ingredient_source_concept_id','moa','gene']]
b = pd.concat([b1,b2])
b = b.loc[b['gene'].notna()].drop_duplicates()
b['moa_priority'] = b['moa'].map({'Negative modulator':2,'Positive modulator':3,'Other':1}).fillna(0)
b = b.sort_values('moa_priority',ascending=False).drop_duplicates(['xtn_generic_ingredient_source_concept_id','gene'])
b = b.sort_values(['xtn_generic_ingredient_source_concept_id'])

a = a.loc[a['gene'].isin(dr['gene'])]
a = a.loc[a['drug_concept_id'].str.len() > 1]

b = b.loc[b['gene'].isin(dr['gene'])]
b = b.loc[b['xtn_generic_ingredient_source_concept_id'].str.len() > 1]

##### Run OMOP query

if False:
    with open('./Conversion/ingredient_concept_id.txt', 'w') as f:
        f.write(f"condition_concept_id IN ({','.join(f'\'{code}\'' for code in b['xtn_generic_ingredient_source_concept_id'].astype(str).unique())})")


In [2]:
if False:
    a1 = pd.read_csv('./OMOP/msdw_drug_exposure_a.csv')
    a2 = pd.read_csv('./OMOP/msdw_med_admin_a.csv')
    
    a1 = pd.concat([a1,a2]).drop_duplicates()
    a2 = 0
    a1 = a1.set_axis(['person_id','concept_id','drug_start','drug_end'],axis=1)
    a1['drug_start'] = pd.to_datetime(a1['drug_start'])
    a1['drug_end'] = pd.to_datetime(a1['drug_end'])
    a1s = a1.groupby(['person_id','concept_id'])['drug_start'].min().reset_index()
    a1e = a1.groupby(['person_id','concept_id'])['drug_end'].max().reset_index()
    a1 = a1s.merge(a1e, on=['person_id','concept_id'], how='outer')
    a1.loc[a1['drug_start'].isna(), 'drug_start'] = a1['drug_end']
    a1.loc[a1['drug_end'].isna(), 'drug_end'] = a1['drug_start']
    a1.to_pickle('./OMOP/msdw_drug_a.pkl')
    a1 = 0

b1 = pd.read_csv('./OMOP/msdw_drug_exposure_b.csv')
b2 = pd.read_csv('./OMOP/msdw_med_admin_b.csv')
b3 = pd.read_csv('./OMOP/msdw_drug_exposure_b_amoxicillin_oseltamivir.csv')
b4 = pd.read_csv('./OMOP/msdw_med_admin_b_amoxicillin_oseltamivir.csv')

b1 = pd.concat([b1,b2,b3,b4]).drop_duplicates()
b2 = 0
b1 = b1.set_axis(['person_id','concept_id','drug_start','drug_end'],axis=1)
b1['drug_start'] = pd.to_datetime(b1['drug_start'], format="%Y-%m-%d")
b1['drug_end'] = pd.to_datetime(b1['drug_end'], format="%Y-%m-%d", errors='coerce')
b1s = b1.groupby(['person_id','concept_id'])['drug_start'].min().reset_index()
b1e = b1.groupby(['person_id','concept_id'])['drug_end'].max().reset_index()
b1 = b1s.merge(b1e, on=['person_id','concept_id'], how='outer')
b1.loc[b1['drug_start'].isna(), 'drug_start'] = b1['drug_end']
b1.loc[b1['drug_end'].isna(), 'drug_end'] = b1['drug_start']
b1.to_pickle('./OMOP/msdw_drug_b.pkl')
b1 = 0


In [7]:
targets = pd.read_pickle('./Resources/targets.pkl')
targets = targets.drop('CID',axis=1).set_axis(['moa','chembl_id','gene','drugbank'],axis=1).drop_duplicates()

dtp = pd.read_pickle('./Resources/drug_target_phase.pkl')
dtp = dtp[['chembl_id','gene','phecode','phase','moa']]
dtp = dtp.loc[dtp['phase'].notna()]
dtp['moa_priority'] = dtp['moa'].map({'Negative modulator':3,'Positive modulator':2,'Other':1}).fillna(0)
dtp = dtp.sort_values(['phase','moa_priority'],ascending=False).drop_duplicates(['chembl_id','gene','phecode'])
dtp = dtp.merge(targets[['chembl_id','drugbank']].dropna().drop_duplicates(), how='left')
dtp = dtp.loc[(dtp['chembl_id'].notna()) | (dtp['drugbank'].notna())]
dtp = dtp[['chembl_id','drugbank','gene','phecode','phase','moa']]

b_conv = pd.read_pickle('./Conversion/name_drugbank_chembl.pkl')
b1 = b_conv.loc[b_conv['drugbank'].notna()].merge(dtp.loc[dtp['drugbank'].notna()].drop('chembl_id',axis=1))
b2 = b_conv.loc[b_conv['chembl_id'].notna()].merge(dtp.loc[dtp['chembl_id'].notna()].drop('drugbank',axis=1))
b_conv = pd.concat([b1,b2])
temp = b_conv[['xtn_generic_ingredient_source_concept_id','dc_name']].drop_duplicates()
b_conv['moa_priority'] = b_conv['moa'].map({'Negative modulator':3,'Positive modulator':2,'Other':1}).fillna(0)
b_conv = b_conv.sort_values(['phase','moa_priority'],ascending=False).drop_duplicates(['dc_name','gene','phecode','moa'])
b_conv = b_conv[['dc_name','gene','phecode','phase','moa']]
b_conv = b_conv.merge(temp, on='dc_name')
b_conv = b_conv.rename({'xtn_generic_ingredient_source_concept_id':'concept_id'},axis=1)
b_conv.to_pickle('./Conversion/concept_id_indications.pkl')

b_conv = pd.read_pickle('./Conversion/name_drugbank_chembl.pkl')
b1 = b_conv.loc[b_conv['drugbank'].notna()].merge(targets.loc[targets['drugbank'].notna()].drop('chembl_id',axis=1))
b2 = b_conv.loc[b_conv['chembl_id'].notna()].merge(targets.loc[targets['chembl_id'].notna()].drop('drugbank',axis=1))
b_conv = pd.concat([b1,b2])
temp = b_conv[['xtn_generic_ingredient_source_concept_id','dc_name']].drop_duplicates()
b_conv['moa_priority'] = b_conv['moa'].map({'Negative modulator':3,'Positive modulator':2,'Other':1}).fillna(0)
b_conv = b_conv.sort_values(['moa_priority'],ascending=False).drop_duplicates(['dc_name','gene','moa'])
b_conv = b_conv[['dc_name','gene','moa']]
b_conv = b_conv.merge(temp, on='dc_name')
b_conv = b_conv.rename({'xtn_generic_ingredient_source_concept_id':'concept_id'},axis=1)
b_conv.to_pickle('./Conversion/concept_id_gene_moa.pkl')


In [8]:
drugs = pd.read_pickle('./Conversion/drug_concept_names.pkl')
ndc = pd.read_pickle('./Conversion/name_drugbank_chembl.pkl')
aid = drugs[['drug_concept_id','drug_concept_code','drug_concept_name']]
aid = aid.loc[~aid['drug_concept_name'].str.contains('no matching concept')]
aid = aid.rename({'drug_concept_name':'dc_name'}, axis=1)
aid = aid.merge(ndc, on='dc_name')[['drug_concept_id','xtn_generic_ingredient_source_concept_id']]
drugs = drugs.loc[~drugs['drug_concept_name'].str.contains('no matching concept|unspecified', case=False)]
drugs = drugs.loc[~drugs['xtn_generic_ingredient_source_concept_name'].str.contains('no matching concept|unspecified', case=False)]
drugs = drugs[['drug_concept_id','xtn_generic_ingredient_source_concept_id']].dropna()
drugs = pd.concat([drugs,aid]).drop_duplicates()
drugs = drugs.loc[(drugs['drug_concept_id'] != '0') & (drugs['xtn_generic_ingredient_source_concept_id'] != 0)]
drugs = drugs.rename({'xtn_generic_ingredient_source_concept_id':'concept_id'},axis=1)
dc = drugs.groupby('concept_id')['drug_concept_id'].nunique().sort_values().reset_index()
dc = dc.loc[dc['drug_concept_id'] < 4]
drugs = drugs.loc[drugs['concept_id'].isin(dc['concept_id'])]

a1 = pd.read_pickle('./OMOP/msdw_drug_a.pkl').drop('drug_end',axis=1).rename({'concept_id':'drug_concept_id'},axis=1)
a1['drug_concept_id'] = a1['drug_concept_id'].astype(str)
a1 = a1.merge(drugs).drop('drug_concept_id',axis=1)

b1 = pd.read_pickle('./OMOP/msdw_drug_b.pkl').drop(['drug_end'],axis=1)
b1['concept_id'] = b1['concept_id'].astype(str)
b1 = pd.concat([b1,a1])
b1 = b1[['person_id','concept_id','drug_start']]
b1 = b1.loc[b1['concept_id'] != '0']
b1 = b1.sort_values('drug_start', ascending=True)
b1 = b1.drop_duplicates(['person_id','concept_id'])
b1.to_pickle('./OMOP/msdw_drug.pkl')


In [9]:
drugs = pd.read_pickle('./OMOP/msdw_drug.pkl')
drugs = drugs.groupby('concept_id')['drug_start'].min().reset_index()
drugs = drugs.rename({'drug_start':'first_dx'},axis=1)
drugs.to_pickle('./OMOP/msdw_drug_first_dx.pkl')


## Other MSDW SQL scripts

In MSDW, prescriptions are stored in two tables: drug_exposure and medication_administration_xtn

In [ ]:
# Get demographic information

SELECT person_id, xtn_birth_date, xtn_vital_status_source_concept_name, gender_concept_code, race_concept_name, race_source_value, ethnicity_concept_name, ethnicity_source_value, xtn_race_ethnicity_source_concept_name, xtn_three_digit_zip FROM cdm_deid.person


In [15]:
# Get relevant drugs

## Part 1
SELECT 
    person_id, 
    drug_concept_id, 
    MIN(drug_exposure_start_date) AS earliest_drug_exposure_start_date, 
    MAX(drug_exposure_end_date) AS latest_drug_exposure_end_date
FROM 
    cdm_deid.drug_exposure
WHERE 
    drug_concept_id IN ('703547', '708298', '711584', '713192', '715233', '718583', '723013', '739323', '740560', '740910', '742267', '744798', '757688', '785649', '789578', '791967', '797399', '798874', '903963', '904351', '904525', '914335', '915175', '923645', '925636', '929887', '929983', '934075', '939259', '939881', '950435', '950637', '954688', '955632', '956874', '963353', '964339', '965748', '970250', '974166', '975125', '978236', '985708', '988095', '989878', '996416', '1000560', '1000632', '1103640', '1105775', '1107830', '1110410', '1110727', '1112807', '1113648', '1115008', '1118084', '1125315', '1126658', '1129625', '1139042', '1139699', '1143374', '1146810', '1151422', '1154343', '1156378', '1177480', '1185922', '1303425', '1304044', '1305058', '1307542', '1308368', '1309068', '1309204', '1314002', '1314924', '1316262', '1318853', '1331247', '1331270', '1336825', '1336926', '1338512', '1340128', '1343916', '1344965', '1348265', '1350310', '1353256', '1354118', '1361711', '1368671', '1377141', '1381504', '1386957', '1398399', '1436650', '1501700', '1504620', '1506270', '1518254', '1520218', '1522957', '1523280', '1524674', '1539403', '1548111', '1548195', '1552310', '1592085', '1636780', '1703653', '1714319', '1738521', '1760039', '1794280', '19002851', '19008264', '19010482', '19010877', '19010963', '19011129', '19012543', '19014878', '19018514', '19019620', '19024227', '19026848', '19034726', '19037833', '19045045', '19047103', '19049038', '19050832', '19054410', '19059135', '19068900', '19069072', '19072024', '19072126', '19090761', '19095164', '19100454', '19100455', '19113014', '19125195', '19134066', '19135908', '37499437', '40163615', '40170680', '40173184', '40244464', '42873638', '42904205', '45892579')
GROUP BY 
    person_id, 
    drug_concept_id;

#####

SELECT 
    person_id, 
    drug_concept_id, 
    MIN(drug_exposure_start_date) AS earliest_drug_exposure_start_date, 
    MAX(drug_exposure_end_date) AS latest_drug_exposure_end_date
FROM 
    cdm_deid.medication_administration_xtn 
WHERE 
    drug_concept_id IN ('703547', '708298', '711584', '713192', '715233', '718583', '723013', '739323', '740560', '740910', '742267', '744798', '757688', '785649', '789578', '791967', '797399', '798874', '903963', '904351', '904525', '914335', '915175', '923645', '925636', '929887', '929983', '934075', '939259', '939881', '950435', '950637', '954688', '955632', '956874', '963353', '964339', '965748', '970250', '974166', '975125', '978236', '985708', '988095', '989878', '996416', '1000560', '1000632', '1103640', '1105775', '1107830', '1110410', '1110727', '1112807', '1113648', '1115008', '1118084', '1125315', '1126658', '1129625', '1139042', '1139699', '1143374', '1146810', '1151422', '1154343', '1156378', '1177480', '1185922', '1303425', '1304044', '1305058', '1307542', '1308368', '1309068', '1309204', '1314002', '1314924', '1316262', '1318853', '1331247', '1331270', '1336825', '1336926', '1338512', '1340128', '1343916', '1344965', '1348265', '1350310', '1353256', '1354118', '1361711', '1368671', '1377141', '1381504', '1386957', '1398399', '1436650', '1501700', '1504620', '1506270', '1518254', '1520218', '1522957', '1523280', '1524674', '1539403', '1548111', '1548195', '1552310', '1592085', '1636780', '1703653', '1714319', '1738521', '1760039', '1794280', '19002851', '19008264', '19010482', '19010877', '19010963', '19011129', '19012543', '19014878', '19018514', '19019620', '19024227', '19026848', '19034726', '19037833', '19045045', '19047103', '19049038', '19050832', '19054410', '19059135', '19068900', '19069072', '19072024', '19072126', '19090761', '19095164', '19100454', '19100455', '19113014', '19125195', '19134066', '19135908', '37499437', '40163615', '40170680', '40173184', '40244464', '42873638', '42904205', '45892579')
GROUP BY 
    person_id, 
    drug_concept_id;

## Part 2
SELECT 
    person_id, 
    xtn_generic_ingredient_source_concept_id , 
    MIN(drug_exposure_start_date) AS earliest_drug_exposure_start_date, 
    MAX(drug_exposure_end_date) AS latest_drug_exposure_end_date
FROM 
    cdm_deid.drug_exposure
WHERE 
    xtn_generic_ingredient_source_concept_id IN ('2000008379','2000008399','2000008416','2000008441','2000008442','2000008443','2000008444','2000008451','2000008452','2000008454','2000008461','2000008462','2000008480','2000008486','2000008489','2000008490','2000008491','2000008530','2000008531','2000008532','2000008542','2000008545','2000008546','2000008548','2000008551','2000008571','2000008572','2000008585','2000008586','2000008597','2000008604','2000008623','2000008627','2000008631','2000008661','2000008667','2000008669','2000008692','2000008698','2000008717','2000008718','2000008720','2000008731','2000008741','2000008744','2000008746','2000008753','2000008756','2000008757','2000008759','2000008781','2000008791','2000008796','2000008801','2000008861','2000008869','2000008879','2000008886','2000008895','2000008899','2000008916','2000008918','2000008925','2000008934','2000008954','2000008970','2000009006','2000009011','2000009035','2000009061','2000009124','2000009138','2000009157','2000009159','2000009185','2000009187','2000009197','2000009204','2000009212','2000009222','2000009223','2000009237','2000009239','2000009255','2000009261','2000009274','2000009282','2000009296','2000009316','2000009361','2000009364','2000009373','2000009374','2000009407','2000009408','2000009452','2000009461','2000009466','2000009470','2000009496','2000009499','2000009502','2000009515','2000009535','2000009536','2000009537','2000009538','2000009542','2000009693','2000009695','2000009721','2000009724','2000009757','2000009765','2000009766','2000009769','2000009775','2000009776','2000009784','2000009785','2000009865','2000009866','2000009888','2000009889','2000009890','2000009891','2000009892','2000009894','2000009895','2000009896','2000009906','2000009915','2000009918','2000009937','2000009982','2000009984','2000009985','2000009986','2000009988','2000009992','2000009994','2000010010','2000010056','2000010057','2000010060','2000010061','2000010062','2000010063','2000010064','2000010066','2000010081','2000010086','2000010087','2000010088','2000010089','2000010090','2000010091','2000010092','2000010093','2000010094','2000010095','2000010096','2000010097','2000010098','2000010103','2000010104','2000010105','2000010106','2000010107','2000010108','2000010109','2000010110','2000010111','2000010112','2000010113','2000010115','2000010116','2000010117','2000010121','2000010221','2000010223','2000010224','2000010225','2000010226','2000010227','2000010228','2000010230','2000010231','2000010232','2000010233','2000010234','2000010235','2000010236','2000010237','2000010238','2000010239','2000010240','2000010241','2000010242','2000010243','2000010244','2000010245','2000010246','2000010247','2000010248','2000010249','2000010250','2000010251','2000010252','2000010253','2000010254','2000010255','2000010256','2000010257','2000010258','2000010259','2000010271','2000010280','2000010288','2000010309','2000010311','2000010312','2000010344','2000010345','2000010346','2000010347','2000010348','2000010350','2000010351','2000010352','2000010353','2000010354','2000010355','2000010356','2000010357','2000010359','2000010360','2000010361','2000010363','2000010364','2000010365','2000010366','2000010367','2000010368','2000010370','2000010371','2000010373','2000010374','2000010375','2000010376','2000010378','2000010380','2000010383','2000010387','2000010388','2000010389','2000010403','2000010404','2000010409','2000010410','2000010411','2000010415','2000010426','2000010427','2000010428','2000010431','2000010500','2000010520','2000010523','2000010537','2000010538','2000010540','2000010541','2000010543','2000010546','2000010547','2000010550','2000010551','2000010552','2000010572','2000010573','2000010589','2000010630','2000010638','2000010645','2000010646','2000010647','2000010648','2000010651','2000010653','2000010654','2000010660','2000010668','2000010773','2000010774','2000010775','2000010776','2000010777','2000010778','2000010783','2000010785','2000010786','2000010788','2000010789','2000010791','2000010793','2000010794','2000010797','2000010798','2000010799','2000010800','2000010802','2000010803','2000010806','2000010808','2000010810','2000010811','2000010813','2000010814','2000010815','2000010816','2000010817','2000010818','2000010822','2000010823','2000010838','2000010839','2000010876','2000010884','2000010891','2000010898','2000010899','2000010903','2000010927','2000010934','2000010938','2000011114','2000011115','2000011124','2000011128','2000011139','2000011143','2000011144','2000011145','2000011151','2000011152','2000011154','2000011155','2000011233','2000011237','2000011242','2000011261','2000011262','2000011264','2000011267','2000011268','2000011270','2000011271','2000011276','2000011277','2000011278','2000011280','2000011281','2000011282','2000011283','2000011285','2000011286','2000011290','2000011291','2000011301','2000011305','2000011376','2000011377','2000011378','2000011399','2000011400','2000011442','2000011449','2000011485','2000011486','2000011487','2000011488','2000011489','2000011490','2000011491','2000011493','2000011494','2000011495','2000011496','2000011497','2000011498','2000011499','2000011500','2000011501','2000011502','2000011503','2000011504','2000011505','2000011507','2000011557','2000011558','2000011560','2000011561','2000011565','2000011566','2000011567','2000011568','2000011569','2000011570','2000011571','2000011572','2000011578','2000011580','2000011584','2000011592','2000011615','2000011626','2000011662','2000011665','2000011674','2000011675','2000011731','2000011742','2000011781','2000011790','2000011793','2000011844','2000011845','2000011846','2000011848','2000011849','2000011851','2000011852','2000011853','2000011854','2000011856','2000011857','2000011859','2000011860','2000011861','2000011862','2000011866','2000011868','2000011871','2000011933','2000011938','2000011939','2000011940','2000011943','2000011976','2000011986','2000011987','2000011989','2000011992','2000011996','2000011997','2000011998','2000012007','2000012014','2000012015','2000012017','2000012044','2000012049','2000012050','2000012051','2000012052','2000012053','2000012055','2000012136','2000012200','2000012202','2000012204','2000012212','2000012214','2000012215','2000012408','2000012413','2000012420','2000012428','2000012436','2000012440','2000012446','2000012448','2000012452','2000012453','2000012454','2000012456','2000012459','2000012460','2000012461','2000012465','2000012467','2000012468','2000012509','2000012602','2000012608','2000012624','2000012625','2000012626','2000012627','2000012628','2000012629','2000012631','2000012634','2000012635','2000012637','2000012638','2000012640','2000012682','2000012683','2000012685','2000012686','2000012687','2000012690','2000012691','2000012692','2000012694','2000012695','2000012696','2000012697','2000012698','2000012699','2000012700','2000012701','2000012702','2000012703','2000012705','2000012706','2000012707','2000012708','2000012709','2000012710','2000012711','2000012718','2000012719','2000012720','2000012722','2000012725','2000012726','2000012768','2000012769','2000012770','2000012772','2000012774','2000012789','2000012790','2000012791','2000012792','2000012793','2000012794','2000012795','2000012796','2000012798','2000012799','2000012804','2000012805','2000012806','2000012808','2000012809','2000012839','2000012891','2000012904','2000012906','2000012931','2000012932','2000012947','2000012948','2000012951','2000013014','2000013062','2000013074','2000013075','2000013076','2000013077','2000013081','2000013082','2000013083','2000013088','2000013089','2000013090','2000013091','2000013095','2000013096','2000013100','2000013101','2000013103','2000013104','2000013105','2000013165','2000013315','2000013380','2000013396','2000013397','2000013398','2000013399','2000013400','2000013401','2000013402','2000013406','2000013407','2000013408','2000013409','2000013412','2000013415','2000013416','2000013417','2000013420','2000013421','2000013422','2000013423','2000013424','2000013425','2000013427','2000013428','2000013429','2000013430','2000013431','2000013432','2000013433','2000013434','2000013435','2000013436','2000013437','2000013438','2000013439','2000013440','2000013441','2000013488','2000013489','2000013490','2000013501','2000013503','2000013509','2000013510','2000013511','2000013512','2000013513','2000013515','2000013517','2000013518','2000013519','2000013520','2000013521','2000013522','2000013524','2000013525','2000013526','2000013527','2000013528','2000013530','2000013584','2000013602','2000013614','2000013615','2000013617','2000013618','2000013619','2000013620','2000013621','2000013622','2000013623','2000013624','2000013625','2000013626','2000013629','2000013630','2000013714','2000013724','2000013743','2000013744','2000013745','2000013746','2000013748','2000013749','2000013759','2000013760','2000013769','2000013770','2000013776','2000013780','2000013784','2000013926','2000013930','2000014045','2000014046','2000014047','2000014048','2000014049','2000014050','2000014051','2000014052','2000014056','2000014057','2000014058','2000014060','2000014061','2000014062','2000014063','2000014064','2000014066','2000014074','2000014076','2000014080','2000014081','2000014102','2000014124','2000014162','2000014306','2000014317','2000014361','2000014397','2000014415','2000014416','2000014450','2000014451','2000014541','2000014544','2000014545','2000014608','2000014848','2000014888','2000014894','2000014984','2000014989','2000014990','2000014993','2000015055','2000015092','2000015133','2000015136','2000015176','2000015245','2000015246','2000015261','2000015322','2000015366','2000015372','2000015635','2000015639','2000015730','2000015734','2000015763','2000015769','2000015817','2000015837','2000015963','2000015976','2000015992','2000016007','2000016066','2000016077','2000016102','2000016112','2000016138','2000016168','2000016175','2000016179','2000016180','2000016201','2000016233','2000016236','2000016238','2000016275','2000016276','2000016416','2000016417','2000016422','2000016460','2000016468','2000016536','2000016546','2000016578','2000016579','2000016581','2000016583','2000016585','2000016586','2000016587','2000016588','2000016589','2000016591','2000016594','2000016597','2000016648','2000016672','2000016676','2000016736','2000016767','2000016769','2000016773','2000016776','2000016813','2000016865','2000017073','2000017086','2000017099','2000017319','2000017404','2000017603','2000017620','2000017640','2000017836','2000017837','2000017847','2000017853','2000017861','2000017891','2000018054','2000018057','2000018058','2000018073','2000018082','2000018180','2000018182','2000018183','2000018184','2000018185','2000018186','2000018187','2000018188','2000018189','2000018200','2000018201','2000018210','2000018212','2000018213','2000018217','2000018218','2000018219','2000018228','2000018266','2000018270','2000018415','2000018445','2000018464','2000018473','2000018474','2000018513','2000018515','2000018519','2000018521','2000018523','2000018525','2000018526','2000018528','2000018530','2000018532','2000018533','2000018534','2000018535','2000018536','2000018541','2000018542','2000018543','2000018547','2000018549','2000018679','2000018749','2000018756','2000018757','2000018766','2000018786','2000018797','2000018798','2000018806','2000018858','2000018870','2000018901','2000018938','2000019013','2000019020','2000019038','2000019040','2000019057','2000019060','2000019156','2000019223','2000019226','2000019239','2000019250','2000019324','2000019326','2000019347','2000019371','2000019372','2000019382','2000019392','2000019401','2000019421','2000019423','2000019475','2000019516','2000019589','2000019590','2000019592','2000019594','2000019596','2000019597','2000019788','2000019824','2000019840','2000019959','2000019961','2000019968','2000019970','2000019981','2000019987','2000020025','2000020085','2000020125','2000020168','2000020182','2000020188','2000020271','2000020281','2000020283','2000020287','2000020302','2000020312','2000020428','2000020527','2000020616','2000020639','2000020641','2000020670','2000020683','2000020685','2000020869','2000020884','2000020890','2000020894','2000020918','2000020924','2000021025','2000021115','2000021121','2000021143','2000021146','2000021171','2000021176','2000021177','2000021205','2000021209','2000021275','2000021291','2000021304','2000021311','2000021332','2000021335','2000021401','2000021405','2000021414','2000021440','2000021469','2000021471','2000021472','2000021502','2000021507','2000021634','2000021644','2000021783','2000021800','2000021814','2000021815','2000021816','2000021847','2000021874','2000021886','2000021904','2000021910','2000021960','2000021984','2000021996','2000021998','2000022019','2000022023','2000022028','2000022029','2000022072','2000022095','2000022112','2000022150','2000022166','2000022167','2000022170','2000022178','2000022187','2000022194','2000022195','2000022205','2000022206','2000022225','2000022236','2000022242','2000022258','2000022260','2000022266','2000022273','2000022274','2000022283','2000022310','2000022326','2000022327','2000022369','2000022383','2000022384','2000022387','2000022392','2000022397','2000022411','2000022423','2000022425','2000022528','2000022544','2000022558','2000022600','2000022628','2000022630','2000022632','2000022646','2000022680','2000022691','2000022701','2000022702','2000022706','2000022730','2000022735','2000022783','2000022802','2000022818','2000022827','2000022866','2000022911','2000022917','2000022926','2000023000','2000023019','2000023039','2000023066','2000023113','2000023127','2000023130','2000023154','2000023157','2000023164','2000023192','2000023196','2000023208','2000023217','2000023263','2000023283','2000023304','2000023306','2000023326','2000023328','2000023329','2000023341','2000023390','2000023393','2000023394','2000023395','2002012123','2002012140','2002012148','2002012161','2002012165','2002012180','2002012182','2002012188','2002012189','2002012193','2002012194','2002012196','2002012199','2002012206','2002012218','2002012225','2002051553','2002051567','2002076847','2002076861','2002076864','2002076878','2002076893','2002076923','2002076932','2002076941','2002076957','2002076967','2002086546','2002087951','2002087957','2002088115','2002088904','2002092914','2002102806','2002102827','2002103848','2002105358','2002106357','2002106360','2002108935','2002108975','2002109903','2002109910','2002109923','2002211833','2002211840','2002221059','2002221081','2002228637','2002235183','2002235188','2002239409','2002239423','2002239424','2002239426','2002248006')
GROUP BY 
    person_id, 
    xtn_generic_ingredient_source_concept_id;

#####

SELECT 
    person_id, 
    xtn_generic_ingredient_source_concept_id , 
    MIN(drug_exposure_start_date) AS earliest_drug_exposure_start_date, 
    MAX(drug_exposure_end_date) AS latest_drug_exposure_end_date
FROM 
    cdm_deid.medication_administration_xtn 
WHERE 
    xtn_generic_ingredient_source_concept_id IN ('2000008379','2000008399','2000008416','2000008441','2000008442','2000008443','2000008444','2000008451','2000008452','2000008454','2000008461','2000008462','2000008480','2000008486','2000008489','2000008490','2000008491','2000008530','2000008531','2000008532','2000008542','2000008545','2000008546','2000008548','2000008551','2000008571','2000008572','2000008585','2000008586','2000008597','2000008604','2000008623','2000008627','2000008631','2000008661','2000008667','2000008669','2000008692','2000008698','2000008717','2000008718','2000008720','2000008731','2000008741','2000008744','2000008746','2000008753','2000008756','2000008757','2000008759','2000008781','2000008791','2000008796','2000008801','2000008861','2000008869','2000008879','2000008886','2000008895','2000008899','2000008916','2000008918','2000008925','2000008934','2000008954','2000008970','2000009006','2000009011','2000009035','2000009061','2000009124','2000009138','2000009157','2000009159','2000009185','2000009187','2000009197','2000009204','2000009212','2000009222','2000009223','2000009237','2000009239','2000009255','2000009261','2000009274','2000009282','2000009296','2000009316','2000009361','2000009364','2000009373','2000009374','2000009407','2000009408','2000009452','2000009461','2000009466','2000009470','2000009496','2000009499','2000009502','2000009515','2000009535','2000009536','2000009537','2000009538','2000009542','2000009693','2000009695','2000009721','2000009724','2000009757','2000009765','2000009766','2000009769','2000009775','2000009776','2000009784','2000009785','2000009865','2000009866','2000009888','2000009889','2000009890','2000009891','2000009892','2000009894','2000009895','2000009896','2000009906','2000009915','2000009918','2000009937','2000009982','2000009984','2000009985','2000009986','2000009988','2000009992','2000009994','2000010010','2000010056','2000010057','2000010060','2000010061','2000010062','2000010063','2000010064','2000010066','2000010081','2000010086','2000010087','2000010088','2000010089','2000010090','2000010091','2000010092','2000010093','2000010094','2000010095','2000010096','2000010097','2000010098','2000010103','2000010104','2000010105','2000010106','2000010107','2000010108','2000010109','2000010110','2000010111','2000010112','2000010113','2000010115','2000010116','2000010117','2000010121','2000010221','2000010223','2000010224','2000010225','2000010226','2000010227','2000010228','2000010230','2000010231','2000010232','2000010233','2000010234','2000010235','2000010236','2000010237','2000010238','2000010239','2000010240','2000010241','2000010242','2000010243','2000010244','2000010245','2000010246','2000010247','2000010248','2000010249','2000010250','2000010251','2000010252','2000010253','2000010254','2000010255','2000010256','2000010257','2000010258','2000010259','2000010271','2000010280','2000010288','2000010309','2000010311','2000010312','2000010344','2000010345','2000010346','2000010347','2000010348','2000010350','2000010351','2000010352','2000010353','2000010354','2000010355','2000010356','2000010357','2000010359','2000010360','2000010361','2000010363','2000010364','2000010365','2000010366','2000010367','2000010368','2000010370','2000010371','2000010373','2000010374','2000010375','2000010376','2000010378','2000010380','2000010383','2000010387','2000010388','2000010389','2000010403','2000010404','2000010409','2000010410','2000010411','2000010415','2000010426','2000010427','2000010428','2000010431','2000010500','2000010520','2000010523','2000010537','2000010538','2000010540','2000010541','2000010543','2000010546','2000010547','2000010550','2000010551','2000010552','2000010572','2000010573','2000010589','2000010630','2000010638','2000010645','2000010646','2000010647','2000010648','2000010651','2000010653','2000010654','2000010660','2000010668','2000010773','2000010774','2000010775','2000010776','2000010777','2000010778','2000010783','2000010785','2000010786','2000010788','2000010789','2000010791','2000010793','2000010794','2000010797','2000010798','2000010799','2000010800','2000010802','2000010803','2000010806','2000010808','2000010810','2000010811','2000010813','2000010814','2000010815','2000010816','2000010817','2000010818','2000010822','2000010823','2000010838','2000010839','2000010876','2000010884','2000010891','2000010898','2000010899','2000010903','2000010927','2000010934','2000010938','2000011114','2000011115','2000011124','2000011128','2000011139','2000011143','2000011144','2000011145','2000011151','2000011152','2000011154','2000011155','2000011233','2000011237','2000011242','2000011261','2000011262','2000011264','2000011267','2000011268','2000011270','2000011271','2000011276','2000011277','2000011278','2000011280','2000011281','2000011282','2000011283','2000011285','2000011286','2000011290','2000011291','2000011301','2000011305','2000011376','2000011377','2000011378','2000011399','2000011400','2000011442','2000011449','2000011485','2000011486','2000011487','2000011488','2000011489','2000011490','2000011491','2000011493','2000011494','2000011495','2000011496','2000011497','2000011498','2000011499','2000011500','2000011501','2000011502','2000011503','2000011504','2000011505','2000011507','2000011557','2000011558','2000011560','2000011561','2000011565','2000011566','2000011567','2000011568','2000011569','2000011570','2000011571','2000011572','2000011578','2000011580','2000011584','2000011592','2000011615','2000011626','2000011662','2000011665','2000011674','2000011675','2000011731','2000011742','2000011781','2000011790','2000011793','2000011844','2000011845','2000011846','2000011848','2000011849','2000011851','2000011852','2000011853','2000011854','2000011856','2000011857','2000011859','2000011860','2000011861','2000011862','2000011866','2000011868','2000011871','2000011933','2000011938','2000011939','2000011940','2000011943','2000011976','2000011986','2000011987','2000011989','2000011992','2000011996','2000011997','2000011998','2000012007','2000012014','2000012015','2000012017','2000012044','2000012049','2000012050','2000012051','2000012052','2000012053','2000012055','2000012136','2000012200','2000012202','2000012204','2000012212','2000012214','2000012215','2000012408','2000012413','2000012420','2000012428','2000012436','2000012440','2000012446','2000012448','2000012452','2000012453','2000012454','2000012456','2000012459','2000012460','2000012461','2000012465','2000012467','2000012468','2000012509','2000012602','2000012608','2000012624','2000012625','2000012626','2000012627','2000012628','2000012629','2000012631','2000012634','2000012635','2000012637','2000012638','2000012640','2000012682','2000012683','2000012685','2000012686','2000012687','2000012690','2000012691','2000012692','2000012694','2000012695','2000012696','2000012697','2000012698','2000012699','2000012700','2000012701','2000012702','2000012703','2000012705','2000012706','2000012707','2000012708','2000012709','2000012710','2000012711','2000012718','2000012719','2000012720','2000012722','2000012725','2000012726','2000012768','2000012769','2000012770','2000012772','2000012774','2000012789','2000012790','2000012791','2000012792','2000012793','2000012794','2000012795','2000012796','2000012798','2000012799','2000012804','2000012805','2000012806','2000012808','2000012809','2000012839','2000012891','2000012904','2000012906','2000012931','2000012932','2000012947','2000012948','2000012951','2000013014','2000013062','2000013074','2000013075','2000013076','2000013077','2000013081','2000013082','2000013083','2000013088','2000013089','2000013090','2000013091','2000013095','2000013096','2000013100','2000013101','2000013103','2000013104','2000013105','2000013165','2000013315','2000013380','2000013396','2000013397','2000013398','2000013399','2000013400','2000013401','2000013402','2000013406','2000013407','2000013408','2000013409','2000013412','2000013415','2000013416','2000013417','2000013420','2000013421','2000013422','2000013423','2000013424','2000013425','2000013427','2000013428','2000013429','2000013430','2000013431','2000013432','2000013433','2000013434','2000013435','2000013436','2000013437','2000013438','2000013439','2000013440','2000013441','2000013488','2000013489','2000013490','2000013501','2000013503','2000013509','2000013510','2000013511','2000013512','2000013513','2000013515','2000013517','2000013518','2000013519','2000013520','2000013521','2000013522','2000013524','2000013525','2000013526','2000013527','2000013528','2000013530','2000013584','2000013602','2000013614','2000013615','2000013617','2000013618','2000013619','2000013620','2000013621','2000013622','2000013623','2000013624','2000013625','2000013626','2000013629','2000013630','2000013714','2000013724','2000013743','2000013744','2000013745','2000013746','2000013748','2000013749','2000013759','2000013760','2000013769','2000013770','2000013776','2000013780','2000013784','2000013926','2000013930','2000014045','2000014046','2000014047','2000014048','2000014049','2000014050','2000014051','2000014052','2000014056','2000014057','2000014058','2000014060','2000014061','2000014062','2000014063','2000014064','2000014066','2000014074','2000014076','2000014080','2000014081','2000014102','2000014124','2000014162','2000014306','2000014317','2000014361','2000014397','2000014415','2000014416','2000014450','2000014451','2000014541','2000014544','2000014545','2000014608','2000014848','2000014888','2000014894','2000014984','2000014989','2000014990','2000014993','2000015055','2000015092','2000015133','2000015136','2000015176','2000015245','2000015246','2000015261','2000015322','2000015366','2000015372','2000015635','2000015639','2000015730','2000015734','2000015763','2000015769','2000015817','2000015837','2000015963','2000015976','2000015992','2000016007','2000016066','2000016077','2000016102','2000016112','2000016138','2000016168','2000016175','2000016179','2000016180','2000016201','2000016233','2000016236','2000016238','2000016275','2000016276','2000016416','2000016417','2000016422','2000016460','2000016468','2000016536','2000016546','2000016578','2000016579','2000016581','2000016583','2000016585','2000016586','2000016587','2000016588','2000016589','2000016591','2000016594','2000016597','2000016648','2000016672','2000016676','2000016736','2000016767','2000016769','2000016773','2000016776','2000016813','2000016865','2000017073','2000017086','2000017099','2000017319','2000017404','2000017603','2000017620','2000017640','2000017836','2000017837','2000017847','2000017853','2000017861','2000017891','2000018054','2000018057','2000018058','2000018073','2000018082','2000018180','2000018182','2000018183','2000018184','2000018185','2000018186','2000018187','2000018188','2000018189','2000018200','2000018201','2000018210','2000018212','2000018213','2000018217','2000018218','2000018219','2000018228','2000018266','2000018270','2000018415','2000018445','2000018464','2000018473','2000018474','2000018513','2000018515','2000018519','2000018521','2000018523','2000018525','2000018526','2000018528','2000018530','2000018532','2000018533','2000018534','2000018535','2000018536','2000018541','2000018542','2000018543','2000018547','2000018549','2000018679','2000018749','2000018756','2000018757','2000018766','2000018786','2000018797','2000018798','2000018806','2000018858','2000018870','2000018901','2000018938','2000019013','2000019020','2000019038','2000019040','2000019057','2000019060','2000019156','2000019223','2000019226','2000019239','2000019250','2000019324','2000019326','2000019347','2000019371','2000019372','2000019382','2000019392','2000019401','2000019421','2000019423','2000019475','2000019516','2000019589','2000019590','2000019592','2000019594','2000019596','2000019597','2000019788','2000019824','2000019840','2000019959','2000019961','2000019968','2000019970','2000019981','2000019987','2000020025','2000020085','2000020125','2000020168','2000020182','2000020188','2000020271','2000020281','2000020283','2000020287','2000020302','2000020312','2000020428','2000020527','2000020616','2000020639','2000020641','2000020670','2000020683','2000020685','2000020869','2000020884','2000020890','2000020894','2000020918','2000020924','2000021025','2000021115','2000021121','2000021143','2000021146','2000021171','2000021176','2000021177','2000021205','2000021209','2000021275','2000021291','2000021304','2000021311','2000021332','2000021335','2000021401','2000021405','2000021414','2000021440','2000021469','2000021471','2000021472','2000021502','2000021507','2000021634','2000021644','2000021783','2000021800','2000021814','2000021815','2000021816','2000021847','2000021874','2000021886','2000021904','2000021910','2000021960','2000021984','2000021996','2000021998','2000022019','2000022023','2000022028','2000022029','2000022072','2000022095','2000022112','2000022150','2000022166','2000022167','2000022170','2000022178','2000022187','2000022194','2000022195','2000022205','2000022206','2000022225','2000022236','2000022242','2000022258','2000022260','2000022266','2000022273','2000022274','2000022283','2000022310','2000022326','2000022327','2000022369','2000022383','2000022384','2000022387','2000022392','2000022397','2000022411','2000022423','2000022425','2000022528','2000022544','2000022558','2000022600','2000022628','2000022630','2000022632','2000022646','2000022680','2000022691','2000022701','2000022702','2000022706','2000022730','2000022735','2000022783','2000022802','2000022818','2000022827','2000022866','2000022911','2000022917','2000022926','2000023000','2000023019','2000023039','2000023066','2000023113','2000023127','2000023130','2000023154','2000023157','2000023164','2000023192','2000023196','2000023208','2000023217','2000023263','2000023283','2000023304','2000023306','2000023326','2000023328','2000023329','2000023341','2000023390','2000023393','2000023394','2000023395','2002012123','2002012140','2002012148','2002012161','2002012165','2002012180','2002012182','2002012188','2002012189','2002012193','2002012194','2002012196','2002012199','2002012206','2002012218','2002012225','2002051553','2002051567','2002076847','2002076861','2002076864','2002076878','2002076893','2002076923','2002076932','2002076941','2002076957','2002076967','2002086546','2002087951','2002087957','2002088115','2002088904','2002092914','2002102806','2002102827','2002103848','2002105358','2002106357','2002106360','2002108935','2002108975','2002109903','2002109910','2002109923','2002211833','2002211840','2002221059','2002221081','2002228637','2002235183','2002235188','2002239409','2002239423','2002239424','2002239426','2002248006')
GROUP BY 
    person_id, 
    xtn_generic_ingredient_source_concept_id;



In [ ]:
# Get earliest prescription for each person

SELECT person_id, 
       MIN(drug_exposure_start_date) AS min_drug_exposure_start_date, 
       MAX(drug_exposure_end_date) AS max_drug_exposure_end_date
FROM (
    SELECT person_id, 
           drug_exposure_start_date, 
           drug_exposure_end_date
    FROM cdm_deid.drug_exposure
    UNION ALL
    SELECT person_id, 
           drug_exposure_start_date, 
           drug_exposure_end_date
    FROM cdm_deid.medication_administration_xtn
) combined
GROUP BY person_id;


In [ ]:
# Get earliest outpatient visit for each person

SELECT person_id, 
       MIN(visit_start_date) AS earliest_visit_start_date, 
       MAX(visit_end_date) AS latest_visit_end_date
FROM cdm_deid.visit_occurrence
WHERE LOWER(xtn_visit_type_source_concept_name) LIKE '%outpatient%'
GROUP BY person_id;


In [ ]:
# Get earliest overall visit for each person

SELECT person_id, 
       MIN(visit_start_date) AS earliest_visit_start_date, 
       MAX(visit_end_date) AS latest_visit_end_date
FROM cdm_deid.visit_occurrence
GROUP BY person_id;
